# Model Hypothesis
1. Insiders perspective:
An insider seeks profit in making insider buyback.
The more the buyback, the higher confidence he has – using his material insider information.

2. Our perspective:
The fundamentals: return, profitability, valuation, risk factors, etc.
The ‘better’ the fundamentals, the more convincing the insider ‘endorsement’ from part 1.

1+2: The ideal stocks (buy signals) appear when those two perspectives and evaluations match. 

In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
import requests
from bs4 import BeautifulSoup

'\nimport pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nfrom sklearn.linear_model import LinearRegression\nfrom sklearn.linear_model import Lasso\nfrom sklearn.linear_model import LassoCV\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.pipeline import make_pipeline\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.metrics import mean_squared_error\nfrom sklearn.decomposition import PCA\nfrom sklearn.impute import SimpleImputer\nimport requests\nfrom bs4 import BeautifulSoup\n'

In [74]:
# load the dataset into Python
df = pd.read_csv('INSIDER_CUSTM_3_26_csv.csv', delimiter=',')

# drop the top 1 blank row & bottom 10 N/A rows
df = df.drop(0)
df = df.drop(df.tail(11).index)
df
# a large pool of S&P tickers
# in demand of data cleaning & ticker selection

,Symbol,Name,Sec Market Value (LOCAL),Com Shs to Calc Diluted EPS,LST Num Shs Purch_0M,LST Num Shs Purch_1M,LST Num Shs Purch_2M,LST Num Shs Purch_3M,LST Num Shs Purch_4M,LST Num Shs Purch_5M,...,Price /Sales CFY,Price / Earns Ratio,Price / Book Value,Entrpr Value/ Sales,Entrpr Value/ EBITDA,Entrpr Value/ EBIT,Div Yield,Total Debt% Equity,EBIT Interest Coverage,Current Ratio
1,FLWS,"1-800-FLOWERS.COM, Inc. Class A",399.608000,65.617000,NaN,0.0,NaN,0.0,0.0,NaN,...,0.285009,21.249723,1.214968,0.406217,8.289439,17.830235,0.0,58.710781,7.524440,1.310658
2,VCXA,10x Capital Venture Acquisition Corp. II Class A,210.693390,27.321667,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.383362,NaN,NaN,NaN,0.0,0.000000,NaN,1.292346
3,TXG,10x Genomics Inc Class A,4879.304700,113.858684,0.0,0.0,NaN,0.0,0.0,NaN,...,8.034349,NaN,5.209733,7.380912,NaN,NaN,0.0,11.812203,-343.966387,4.849187
4,YI,"111, Inc. Sponsored ADR Class A",131.150280,83.317060,NaN,NaN,NaN,NaN,NaN,NaN,...,0.125350,NaN,NaN,0.155423,NaN,NaN,NaN,NaN,-27.110394,1.187411
5,YQ,"17 Education & Technology Group, Inc. Unsponso...",47.474472,49.405570,NaN,NaN,NaN,NaN,NaN,NaN,...,0.182325,NaN,0.421535,-0.296504,NaN,NaN,0.0,18.470182,NaN,1.965760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5098,CNET,ZW Data Action Technologies Inc,14.474381,6.609784,NaN,NaN,NaN,NaN,NaN,NaN,...,0.698295,NaN,1.623298,0.597404,NaN,NaN,0.0,11.017183,NaN,2.258021
5099,ZYME,Zymeworks Inc,525.138500,65.249184,0.0,NaN,2318900.0,0.0,0.0,0.0,...,1.243348,4.132275,1.028174,0.118201,0.137113,0.148916,NaN,5.706189,NaN,5.696272
5100,ZYNE,Zynerba Pharmaceuticals Inc,20.246380,39.259495,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,1.673961,NaN,NaN,NaN,0.0,0.793591,NaN,8.085458
5101,ZYXI,"Zynex, Inc.",402.612730,39.127000,0.0,0.0,0.0,0.0,0.0,0.0,...,3.441025,31.925637,7.719286,3.484112,19.916418,23.326121,0.0,40.626601,52.131818,3.298824


In [75]:
# get the ticker list that is aligned with the other models
# screened by market cap & trading volume
# from website: finviz

def get_candidates():
    offset = 0
    link_fmt = 'https://finviz.com/screener.ashx?v=111&f=cap_mega,geo_usa,sh_curvol_o1000&r={offset}'
    tickers = list()

    while True:
        url = link_fmt.format(offset=offset)
        response = requests.get(
            url,
            headers= {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
            },
            params={
                'referer':'https://google.com/',
                'sec-ch-ua-platform':"Windows",
                'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
                'authority':'apkmirror.com',
                'cache-control':'no-cache',
                'sec-fetch-dest':'document',
                'accept-language':'en-US,en;q=0.9',
                'sec-fetch-mode':'navigate',
            }, 
        )

        if response.status_code != 200:
            raise Exception(f'Connection error! Status code: {response.status_code}')

        content = response.content
        soup = BeautifulSoup(content)

        table = soup.find('table', attrs={'id':'screener-views-table'}).find_all('a', attrs={'class':'screener-link-primary'})
        if len(table) == 1:
            break
        for item in table:
            tickers.append(item.text)

        offset = len(tickers)

    tickers = list(set(tickers))
    print(len(tickers))
    return tickers

candidates = get_candidates()
# candidates = ['PG', 'META', 'PFE', 'V', 'KO', 'GOOG', 'WMT', 'AAPL', 'CVX', 'TSLA', 'GOOGL', 'JPM', 'ORCL', 'JNJ', 'NVDA', 'BAC', 'AMZN', 'MSFT', 'XOM', 'MRK']
df = df[df['Symbol'].isin(candidates)]
df = df.reset_index(drop=True)
df

31


,Symbol,Name,Sec Market Value (LOCAL),Com Shs to Calc Diluted EPS,LST Num Shs Purch_0M,LST Num Shs Purch_1M,LST Num Shs Purch_2M,LST Num Shs Purch_3M,LST Num Shs Purch_4M,LST Num Shs Purch_5M,...,Price /Sales CFY,Price / Earns Ratio,Price / Book Value,Entrpr Value/ Sales,Entrpr Value/ EBITDA,Entrpr Value/ EBIT,Div Yield,Total Debt% Equity,EBIT Interest Coverage,Current Ratio
0,ABBV,"AbbVie, Inc.",279600.60,1778.000,0.0,0.0,NaN,0.0,NaN,0.0,...,4.949574,24.388072,16.571080,5.896882,10.989011,15.105636,3.533197,372.035470,9.634271,0.963606
1,GOOGL,Alphabet Inc. Class A,628000.60,14046.000,0.0,0.0,0.0,0.0,0.0,0.0,...,4.412207,20.664215,4.425898,4.113907,12.842342,15.649011,0.000000,11.703183,150.270103,2.377994
2,AMZN,"Amazon.com, Inc.",1005563.56,10189.000,0.0,0.0,NaN,NaN,0.0,0.0,...,1.665183,NaN,5.890921,1.829743,22.321704,64.496854,0.000000,106.113953,5.708069,0.944644
3,AAPL,Apple Inc.,2535466.80,16325.819,0.0,0.0,NaN,NaN,0.0,0.0,...,6.228045,24.607405,47.331220,6.441513,19.370150,21.170984,0.598285,261.446164,40.749574,0.879356
4,BAC,Bank of America Corp,217224.19,8167.500,0.0,0.0,0.0,0.0,0.0,0.0,...,2.341125,10.398091,1.081917,6.728737,NaN,NaN,2.596618,186.278766,NaN,0.766888
5,AVGO,Broadcom Inc.,265234.30,423.000,0.0,NaN,NaN,0.0,0.0,0.0,...,6.024657,17.823827,8.704576,6.854763,11.786399,15.898068,3.467964,176.044740,8.225101,2.623936
6,CVX,Chevron Corporation,297555.56,1940.000,0.0,0.0,0.0,0.0,0.0,0.0,...,1.473172,9.818445,2.158199,1.514628,6.173651,8.533306,3.164522,17.183360,65.844444,1.471673
7,KO,Coca-Cola Company,263495.56,4350.000,0.0,0.0,0.0,0.0,0.0,33200.0,...,6.458848,28.997994,11.421030,7.175288,23.154658,25.595457,2.766861,168.442232,13.882695,1.145356
8,COST,Costco Wholesale Corporation,219643.94,444.757,0.0,NaN,0.0,0.0,0.0,0.0,...,1.042197,40.473981,11.404780,1.043739,23.204734,29.984816,0.635553,55.198140,49.322785,1.021814
9,LLY,Eli Lilly and Company,319423.03,904.619,0.0,0.0,0.0,0.0,0.0,0.0,...,11.595290,52.995712,32.640480,12.091145,35.203686,41.676515,1.071507,152.477981,24.971049,1.052298


# Insider Buying attributes:
Sec Market Value (LOCAL), Com Shs to Calc Diluted EPS, LST Num Shs Purch_0M, LST Num Shs Purch_1M, LST Num Shs Purch_2M, LST Num Shs Purch_3M, LST Num Shs Purch_4M, LST Num Shs Purch_5M, LST Num Shs Purch_6M

# Return attributes:
Closing Price_latest, Closing Price_5D, Closing Price_1M, Closing Price_3M, Closing Price_6M, Closing Price_1Y

# Profitability attributes:
EBITDA Margin, EBIT Margin, Net Income Margin, ROA, ROE, ROIC, Asset Turnover

# Valuation attributes:
Price /Sales CFY, Price / Earns Ratio, Price / Book Value, Entrpr Value/ Sales, Entrpr Value/ EBITDA, Entrpr Value/ EBIT

# Management & Risk factors attributes:
Div Yield, Total Debt% Equity, EBIT Interest Coverage, Current Ratio

In [76]:
# Clean the dataset

# 1. drop rows that contain missing values of 'insider buying' & 'return'
df.dropna(subset=['Com Shs to Calc Diluted EPS'], inplace=True)
df.dropna(subset=['Close Price_latest', 'Close Price_1M', 'Close Price_3M', 'Close Price_6M', 'Close Price_1Y'], inplace=True)

# 2. drop rows that have closing price of zero
df = df[(df[['Close Price_latest', 'Close Price_1M', 'Close Price_3M', 'Close Price_6M', 'Close Price_1Y']] != 0).all(axis=1)]
df


,Symbol,Name,Sec Market Value (LOCAL),Com Shs to Calc Diluted EPS,LST Num Shs Purch_0M,LST Num Shs Purch_1M,LST Num Shs Purch_2M,LST Num Shs Purch_3M,LST Num Shs Purch_4M,LST Num Shs Purch_5M,...,Price /Sales CFY,Price / Earns Ratio,Price / Book Value,Entrpr Value/ Sales,Entrpr Value/ EBITDA,Entrpr Value/ EBIT,Div Yield,Total Debt% Equity,EBIT Interest Coverage,Current Ratio
0,ABBV,"AbbVie, Inc.",279600.60,1778.000,0.0,0.0,NaN,0.0,NaN,0.0,...,4.949574,24.388072,16.571080,5.896882,10.989011,15.105636,3.533197,372.035470,9.634271,0.963606
1,GOOGL,Alphabet Inc. Class A,628000.60,14046.000,0.0,0.0,0.0,0.0,0.0,0.0,...,4.412207,20.664215,4.425898,4.113907,12.842342,15.649011,0.000000,11.703183,150.270103,2.377994
2,AMZN,"Amazon.com, Inc.",1005563.56,10189.000,0.0,0.0,NaN,NaN,0.0,0.0,...,1.665183,NaN,5.890921,1.829743,22.321704,64.496854,0.000000,106.113953,5.708069,0.944644
3,AAPL,Apple Inc.,2535466.80,16325.819,0.0,0.0,NaN,NaN,0.0,0.0,...,6.228045,24.607405,47.331220,6.441513,19.370150,21.170984,0.598285,261.446164,40.749574,0.879356
4,BAC,Bank of America Corp,217224.19,8167.500,0.0,0.0,0.0,0.0,0.0,0.0,...,2.341125,10.398091,1.081917,6.728737,NaN,NaN,2.596618,186.278766,NaN,0.766888
5,AVGO,Broadcom Inc.,265234.30,423.000,0.0,NaN,NaN,0.0,0.0,0.0,...,6.024657,17.823827,8.704576,6.854763,11.786399,15.898068,3.467964,176.044740,8.225101,2.623936
6,CVX,Chevron Corporation,297555.56,1940.000,0.0,0.0,0.0,0.0,0.0,0.0,...,1.473172,9.818445,2.158199,1.514628,6.173651,8.533306,3.164522,17.183360,65.844444,1.471673
7,KO,Coca-Cola Company,263495.56,4350.000,0.0,0.0,0.0,0.0,0.0,33200.0,...,6.458848,28.997994,11.421030,7.175288,23.154658,25.595457,2.766861,168.442232,13.882695,1.145356
8,COST,Costco Wholesale Corporation,219643.94,444.757,0.0,NaN,0.0,0.0,0.0,0.0,...,1.042197,40.473981,11.404780,1.043739,23.204734,29.984816,0.635553,55.198140,49.322785,1.021814
9,LLY,Eli Lilly and Company,319423.03,904.619,0.0,0.0,0.0,0.0,0.0,0.0,...,11.595290,52.995712,32.640480,12.091145,35.203686,41.676515,1.071507,152.477981,24.971049,1.052298


In [77]:
# 3. convert the unit of 'millions'
df["Com Shs to Calc Diluted EPS"] = df["Com Shs to Calc Diluted EPS"] * 1000000
df["Sec Market Value (LOCAL)"] = df["Sec Market Value (LOCAL)"] * 1000000

In [78]:
# 4. replace with zero for missing 'LST Num Shs Purch'
df[['LST Num Shs Purch_0M', 'LST Num Shs Purch_1M', 'LST Num Shs Purch_2M', 'LST Num Shs Purch_3M', 'LST Num Shs Purch_4M', 'LST Num Shs Purch_5M', 'LST Num Shs Purch_6M']] = df [['LST Num Shs Purch_0M', 'LST Num Shs Purch_1M', 'LST Num Shs Purch_2M', 'LST Num Shs Purch_3M', 'LST Num Shs Purch_4M', 'LST Num Shs Purch_5M', 'LST Num Shs Purch_6M']].fillna(0)

# add them up to get ALL insider purchase within the last 6 months
df['Total Insider Purchases'] = df[['LST Num Shs Purch_0M', 'LST Num Shs Purch_1M', 'LST Num Shs Purch_2M', 
                                    'LST Num Shs Purch_3M', 'LST Num Shs Purch_4M', 'LST Num Shs Purch_5M', 
                                    'LST Num Shs Purch_6M']].sum(axis=1)

df = df.drop(['LST Num Shs Purch_0M', 'LST Num Shs Purch_1M', 'LST Num Shs Purch_2M', 'LST Num Shs Purch_3M', 'LST Num Shs Purch_4M', 'LST Num Shs Purch_5M', 'LST Num Shs Purch_6M'], axis=1)


In [79]:
df['Total Insider Purchases'].describe()

count       29.000000
mean      1462.172414
std       6254.644109
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max      33200.000000
Name: Total Insider Purchases, dtype: float64

In [80]:
# 5. deal with untransparent stocks 
# drop the rows which have more than 70% missing values
threshold = len(df.columns) * 0.7  # set threshold to 70% of total columns
df = df.dropna(thresh=threshold)
df

,Symbol,Name,Sec Market Value (LOCAL),Com Shs to Calc Diluted EPS,Close Price_latest,Close Price_1M,Close Price_3M,Close Price_6M,Close Price_1Y,EBITDA Margin,...,Price / Earns Ratio,Price / Book Value,Entrpr Value/ Sales,Entrpr Value/ EBITDA,Entrpr Value/ EBIT,Div Yield,Total Debt% Equity,EBIT Interest Coverage,Current Ratio,Total Insider Purchases
0,ABBV,"AbbVie, Inc.",2.796006e+11,1.778000e+09,158.02,152.71,163.10,143.06,161.3300,48.393090,...,24.388072,16.571080,5.896882,10.989011,15.105636,3.533197,372.035470,9.634271,0.963606,0.0
1,GOOGL,Alphabet Inc. Class A,6.280006e+11,1.404600e+10,105.44,89.13,89.23,98.74,141.6730,38.784360,...,20.664215,4.425898,4.113907,12.842342,15.649011,0.000000,11.703183,150.270103,2.377994,0.0
2,AMZN,"Amazon.com, Inc.",1.005564e+12,1.018900e+10,98.13,93.50,85.25,113.78,164.7735,14.952610,...,NaN,5.890921,1.829743,22.321704,64.496854,0.000000,106.113953,5.708069,0.944644,0.0
3,AAPL,Apple Inc.,2.535467e+12,1.632582e+10,160.25,146.71,131.86,150.43,174.7200,32.297360,...,24.607405,47.331220,6.441513,19.370150,21.170984,0.598285,261.446164,40.749574,0.879356,0.0
4,BAC,Bank of America Corp,2.172242e+11,8.167500e+09,27.14,34.21,32.47,31.73,43.7300,NaN,...,10.398091,1.081917,6.728737,NaN,NaN,2.596618,186.278766,NaN,0.766888,0.0
5,AVGO,Broadcom Inc.,2.652343e+11,4.230000e+08,636.17,577.75,552.43,468.68,628.8700,62.773720,...,17.823827,8.704576,6.854763,11.786399,15.898068,3.467964,176.044740,8.225101,2.623936,0.0
6,CVX,Chevron Corporation,2.975556e+11,1.940000e+09,156.06,162.41,177.40,144.77,169.3100,27.545810,...,9.818445,2.158199,1.514628,6.173651,8.533306,3.164522,17.183360,65.844444,1.471673,0.0
7,KO,Coca-Cola Company,2.634956e+11,4.350000e+09,60.90,59.84,63.82,58.60,61.5300,32.313390,...,28.997994,11.421030,7.175288,23.154658,25.595457,2.766861,168.442232,13.882695,1.145356,33200.0
8,COST,Costco Wholesale Corporation,2.196439e+11,4.447570e+08,495.27,488.61,462.65,466.40,555.7100,4.406260,...,40.473981,11.404780,1.043739,23.204734,29.984816,0.635553,55.198140,49.322785,1.021814,0.0
9,LLY,Eli Lilly and Company,3.194230e+11,9.046190e+08,336.13,321.64,367.90,311.46,289.0200,32.933760,...,52.995712,32.640480,12.091145,35.203686,41.676515,1.071507,152.477981,24.971049,1.052298,0.0


In [81]:
# 6. drop the columns which have more than 40% missing values
# what metrics were dropped?
dropped_cols = []
for col in df.columns:
    if df[col].isna().sum() > len(df)*0.4:
        df.drop(col, axis=1, inplace=True)
        dropped_cols.append(col)

print("Dropped columns:", dropped_cols)
# dropped columns (metrics): 

Dropped columns: []


In [82]:
# Calculate the percentage of missing values in each column
missing_perc = df.isna().mean() * 100

# Get the column names where the percentage of missing values is greater than 40%
columns_to_drop = missing_perc[missing_perc > 40].index

# Drop the columns from the dataframe
df = df.drop(columns_to_drop, axis=1)

In [83]:
df.reset_index(drop=True, inplace=True)
df

,Symbol,Name,Sec Market Value (LOCAL),Com Shs to Calc Diluted EPS,Close Price_latest,Close Price_1M,Close Price_3M,Close Price_6M,Close Price_1Y,EBITDA Margin,...,Price / Earns Ratio,Price / Book Value,Entrpr Value/ Sales,Entrpr Value/ EBITDA,Entrpr Value/ EBIT,Div Yield,Total Debt% Equity,EBIT Interest Coverage,Current Ratio,Total Insider Purchases
0,ABBV,"AbbVie, Inc.",2.796006e+11,1.778000e+09,158.02,152.71,163.10,143.06,161.3300,48.393090,...,24.388072,16.571080,5.896882,10.989011,15.105636,3.533197,372.035470,9.634271,0.963606,0.0
1,GOOGL,Alphabet Inc. Class A,6.280006e+11,1.404600e+10,105.44,89.13,89.23,98.74,141.6730,38.784360,...,20.664215,4.425898,4.113907,12.842342,15.649011,0.000000,11.703183,150.270103,2.377994,0.0
2,AMZN,"Amazon.com, Inc.",1.005564e+12,1.018900e+10,98.13,93.50,85.25,113.78,164.7735,14.952610,...,NaN,5.890921,1.829743,22.321704,64.496854,0.000000,106.113953,5.708069,0.944644,0.0
3,AAPL,Apple Inc.,2.535467e+12,1.632582e+10,160.25,146.71,131.86,150.43,174.7200,32.297360,...,24.607405,47.331220,6.441513,19.370150,21.170984,0.598285,261.446164,40.749574,0.879356,0.0
4,BAC,Bank of America Corp,2.172242e+11,8.167500e+09,27.14,34.21,32.47,31.73,43.7300,NaN,...,10.398091,1.081917,6.728737,NaN,NaN,2.596618,186.278766,NaN,0.766888,0.0
5,AVGO,Broadcom Inc.,2.652343e+11,4.230000e+08,636.17,577.75,552.43,468.68,628.8700,62.773720,...,17.823827,8.704576,6.854763,11.786399,15.898068,3.467964,176.044740,8.225101,2.623936,0.0
6,CVX,Chevron Corporation,2.975556e+11,1.940000e+09,156.06,162.41,177.40,144.77,169.3100,27.545810,...,9.818445,2.158199,1.514628,6.173651,8.533306,3.164522,17.183360,65.844444,1.471673,0.0
7,KO,Coca-Cola Company,2.634956e+11,4.350000e+09,60.90,59.84,63.82,58.60,61.5300,32.313390,...,28.997994,11.421030,7.175288,23.154658,25.595457,2.766861,168.442232,13.882695,1.145356,33200.0
8,COST,Costco Wholesale Corporation,2.196439e+11,4.447570e+08,495.27,488.61,462.65,466.40,555.7100,4.406260,...,40.473981,11.404780,1.043739,23.204734,29.984816,0.635553,55.198140,49.322785,1.021814,0.0
9,LLY,Eli Lilly and Company,3.194230e+11,9.046190e+08,336.13,321.64,367.90,311.46,289.0200,32.933760,...,52.995712,32.640480,12.091145,35.203686,41.676515,1.071507,152.477981,24.971049,1.052298,0.0


In [84]:
# 7.* impute missing values with median
imputer = SimpleImputer(strategy='median')
cols_to_impute = df.columns[2:]
df_imputed = pd.DataFrame(imputer.fit_transform(df[cols_to_impute]), columns=cols_to_impute)

# add back the columns of symbol & name
df_imputed[['Symbol', 'Name']] = df[['Symbol', 'Name']]

# Move Symbol and Name back to the leftmost position & check the resulting dataframe
cols = ['Symbol', 'Name'] + [col for col in df_imputed.columns if col not in ['Symbol', 'Name']]
df_imp = df_imputed[cols]
df_imp


,Symbol,Name,Sec Market Value (LOCAL),Com Shs to Calc Diluted EPS,Close Price_latest,Close Price_1M,Close Price_3M,Close Price_6M,Close Price_1Y,EBITDA Margin,...,Price / Earns Ratio,Price / Book Value,Entrpr Value/ Sales,Entrpr Value/ EBITDA,Entrpr Value/ EBIT,Div Yield,Total Debt% Equity,EBIT Interest Coverage,Current Ratio,Total Insider Purchases
0,ABBV,"AbbVie, Inc.",2.796006e+11,1.778000e+09,158.02,152.71,163.10,143.06,161.3300,48.393090,...,24.388072,16.571080,5.896882,10.989011,15.105636,3.533197,372.035470,9.634271,0.963606,0.0
1,GOOGL,Alphabet Inc. Class A,6.280006e+11,1.404600e+10,105.44,89.13,89.23,98.74,141.6730,38.784360,...,20.664215,4.425898,4.113907,12.842342,15.649011,0.000000,11.703183,150.270103,2.377994,0.0
2,AMZN,"Amazon.com, Inc.",1.005564e+12,1.018900e+10,98.13,93.50,85.25,113.78,164.7735,14.952610,...,25.485311,5.890921,1.829743,22.321704,64.496854,0.000000,106.113953,5.708069,0.944644,0.0
3,AAPL,Apple Inc.,2.535467e+12,1.632582e+10,160.25,146.71,131.86,150.43,174.7200,32.297360,...,24.607405,47.331220,6.441513,19.370150,21.170984,0.598285,261.446164,40.749574,0.879356,0.0
4,BAC,Bank of America Corp,2.172242e+11,8.167500e+09,27.14,34.21,32.47,31.73,43.7300,32.933760,...,10.398091,1.081917,6.728737,18.653166,20.406273,2.596618,186.278766,26.015632,0.766888,0.0
5,AVGO,Broadcom Inc.,2.652343e+11,4.230000e+08,636.17,577.75,552.43,468.68,628.8700,62.773720,...,17.823827,8.704576,6.854763,11.786399,15.898068,3.467964,176.044740,8.225101,2.623936,0.0
6,CVX,Chevron Corporation,2.975556e+11,1.940000e+09,156.06,162.41,177.40,144.77,169.3100,27.545810,...,9.818445,2.158199,1.514628,6.173651,8.533306,3.164522,17.183360,65.844444,1.471673,0.0
7,KO,Coca-Cola Company,2.634956e+11,4.350000e+09,60.90,59.84,63.82,58.60,61.5300,32.313390,...,28.997994,11.421030,7.175288,23.154658,25.595457,2.766861,168.442232,13.882695,1.145356,33200.0
8,COST,Costco Wholesale Corporation,2.196439e+11,4.447570e+08,495.27,488.61,462.65,466.40,555.7100,4.406260,...,40.473981,11.404780,1.043739,23.204734,29.984816,0.635553,55.198140,49.322785,1.021814,0.0
9,LLY,Eli Lilly and Company,3.194230e+11,9.046190e+08,336.13,321.64,367.90,311.46,289.0200,32.933760,...,52.995712,32.640480,12.091145,35.203686,41.676515,1.071507,152.477981,24.971049,1.052298,0.0


In [85]:
df_imp.describe()

,Sec Market Value (LOCAL),Com Shs to Calc Diluted EPS,Close Price_latest,Close Price_1M,Close Price_3M,Close Price_6M,Close Price_1Y,EBITDA Margin,EBIT Margin,Net Income Margin,...,Price / Earns Ratio,Price / Book Value,Entrpr Value/ Sales,Entrpr Value/ EBITDA,Entrpr Value/ EBIT,Div Yield,Total Debt% Equity,EBIT Interest Coverage,Current Ratio,Total Insider Purchases
count,2.900000e+01,2.900000e+01,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,...,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000
mean,5.024745e+11,3.743793e+09,221.429655,215.228621,210.566897,199.533793,235.117466,34.356664,29.919123,20.267102,...,27.752387,18.249333,6.003530,18.777621,23.992554,1.688270,215.560397,37.104383,1.387431,1462.172414
std,5.335368e+11,3.973740e+09,155.671900,148.541715,151.361300,140.398214,160.120050,16.965361,16.974975,11.673285,...,19.856170,38.242824,4.460842,12.023225,17.007565,1.221212,585.357313,43.703858,0.630215,6254.644109
min,1.984771e+11,3.940000e+08,27.140000,34.210000,32.470000,31.730000,43.730000,4.406260,3.481348,-0.529590,...,8.321011,1.081917,0.737681,5.394445,7.375025,0.000000,11.703183,5.708069,0.654545,0.000000
25%,2.634956e+11,1.025000e+09,105.440000,110.750000,111.860000,109.140000,141.920000,22.406270,16.032910,15.004146,...,18.977337,4.889955,2.670503,12.842342,15.898068,0.598285,53.329254,13.882695,0.963606,0.000000
50%,3.325446e+11,2.542000e+09,160.250000,162.410000,152.620000,144.770000,174.720000,32.933760,29.148350,19.895548,...,25.485311,7.524254,5.144527,18.653166,20.406273,1.779755,76.829698,26.015632,1.249013,0.000000
75%,4.440257e+11,4.205000e+09,280.570000,263.680000,267.570000,270.940000,303.680000,47.800600,40.848040,24.830264,...,31.218962,11.510480,6.854763,20.417761,25.595457,2.596618,176.044740,40.418323,1.483186,0.000000
max,2.535467e+12,1.632582e+10,636.170000,577.750000,552.430000,520.790000,628.870000,70.311710,67.831840,49.914705,...,116.885720,205.990600,18.880608,71.367438,91.125610,3.533197,3224.327785,209.718750,3.515618,33200.000000


In [86]:
# create a new DataFrame to record our attribute-group metrics
df_2 = pd.DataFrame({
    'Symbol': df['Symbol'],
    'Name': df['Name'],
    'Insider Buying' : df_imp['Total Insider Purchases'] / df_imp['Com Shs to Calc Diluted EPS'] * 100000
})

df_2

,Symbol,Name,Insider Buying
0,ABBV,"AbbVie, Inc.",0.000000
1,GOOGL,Alphabet Inc. Class A,0.000000
2,AMZN,"Amazon.com, Inc.",0.000000
3,AAPL,Apple Inc.,0.000000
4,BAC,Bank of America Corp,0.000000
5,AVGO,Broadcom Inc.,0.000000
6,CVX,Chevron Corporation,0.000000
7,KO,Coca-Cola Company,0.763218
8,COST,Costco Wholesale Corporation,0.000000
9,LLY,Eli Lilly and Company,0.000000


In [87]:
# calculate returns for different time periods
df_imp['Return_1M'] = (df_imp['Close Price_latest'] - df_imp['Close Price_1M']) / df['Close Price_1M']
df_imp['Return_3M'] = (df_imp['Close Price_latest'] - df_imp['Close Price_3M']) / df['Close Price_3M']
df_imp['Return_6M'] = (df_imp['Close Price_latest'] - df_imp['Close Price_6M']) / df['Close Price_6M']
df_imp['Return_1Y'] = (df_imp['Close Price_latest'] - df_imp['Close Price_1Y']) / df['Close Price_1Y']
df_imp

C:\Users\l3047\AppData\Local\Temp\ipykernel_8268\4052276547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_imp['Return_1M'] = (df_imp['Close Price_latest'] - df_imp['Close Price_1M']) / df['Close Price_1M']
C:\Users\l3047\AppData\Local\Temp\ipykernel_8268\4052276547.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_imp['Return_3M'] = (df_imp['Close Price_latest'] - df_imp['Close Price_3M']) / df['Close Price_3M']
C:\Users\l3047\AppData\Local\Temp\ipykernel_8268\4052276547.py:4: SettingWithCopyW

,Symbol,Name,Sec Market Value (LOCAL),Com Shs to Calc Diluted EPS,Close Price_latest,Close Price_1M,Close Price_3M,Close Price_6M,Close Price_1Y,EBITDA Margin,...,Entrpr Value/ EBIT,Div Yield,Total Debt% Equity,EBIT Interest Coverage,Current Ratio,Total Insider Purchases,Return_1M,Return_3M,Return_6M,Return_1Y
0,ABBV,"AbbVie, Inc.",2.796006e+11,1.778000e+09,158.02,152.71,163.10,143.06,161.3300,48.393090,...,15.105636,3.533197,372.035470,9.634271,0.963606,0.0,0.034772,-0.031147,0.104572,-0.020517
1,GOOGL,Alphabet Inc. Class A,6.280006e+11,1.404600e+10,105.44,89.13,89.23,98.74,141.6730,38.784360,...,15.649011,0.000000,11.703183,150.270103,2.377994,0.0,0.182991,0.181665,0.067855,-0.255751
2,AMZN,"Amazon.com, Inc.",1.005564e+12,1.018900e+10,98.13,93.50,85.25,113.78,164.7735,14.952610,...,64.496854,0.000000,106.113953,5.708069,0.944644,0.0,0.049519,0.151085,-0.137546,-0.404455
3,AAPL,Apple Inc.,2.535467e+12,1.632582e+10,160.25,146.71,131.86,150.43,174.7200,32.297360,...,21.170984,0.598285,261.446164,40.749574,0.879356,0.0,0.092291,0.215304,0.065280,-0.082818
4,BAC,Bank of America Corp,2.172242e+11,8.167500e+09,27.14,34.21,32.47,31.73,43.7300,32.933760,...,20.406273,2.596618,186.278766,26.015632,0.766888,0.0,-0.206665,-0.164152,-0.144658,-0.379373
5,AVGO,Broadcom Inc.,2.652343e+11,4.230000e+08,636.17,577.75,552.43,468.68,628.8700,62.773720,...,15.898068,3.467964,176.044740,8.225101,2.623936,0.0,0.101116,0.151585,0.357365,0.011608
6,CVX,Chevron Corporation,2.975556e+11,1.940000e+09,156.06,162.41,177.40,144.77,169.3100,27.545810,...,8.533306,3.164522,17.183360,65.844444,1.471673,0.0,-0.039099,-0.120293,0.077986,-0.078259
7,KO,Coca-Cola Company,2.634956e+11,4.350000e+09,60.90,59.84,63.82,58.60,61.5300,32.313390,...,25.595457,2.766861,168.442232,13.882695,1.145356,33200.0,0.017714,-0.045754,0.039249,-0.010239
8,COST,Costco Wholesale Corporation,2.196439e+11,4.447570e+08,495.27,488.61,462.65,466.40,555.7100,4.406260,...,29.984816,0.635553,55.198140,49.322785,1.021814,0.0,0.013631,0.070507,0.061900,-0.108762
9,LLY,Eli Lilly and Company,3.194230e+11,9.046190e+08,336.13,321.64,367.90,311.46,289.0200,32.933760,...,41.676515,1.071507,152.477981,24.971049,1.052298,0.0,0.045050,-0.086355,0.079208,0.162999


# Return

In [88]:
# select the return attributes
return_df = df_imp[['Return_1M', 'Return_3M', 'Return_6M', 'Return_1Y']]

# standardize the data
scaler = StandardScaler()
return_df_scaled = scaler.fit_transform(return_df)
pca = PCA(n_components=1)

# perform PCA on the standardized data
pca = PCA(n_components=1)
return_pca = pca.fit_transform(return_df_scaled)

df_2['Return'] = return_pca.flatten()
df_2

,Symbol,Name,Insider Buying,Return
0,ABBV,"AbbVie, Inc.",0.000000,-0.130704
1,GOOGL,Alphabet Inc. Class A,0.000000,1.067291
2,AMZN,"Amazon.com, Inc.",0.000000,-0.529775
3,AAPL,Apple Inc.,0.000000,0.698126
4,BAC,Bank of America Corp,0.000000,-3.012115
5,AVGO,Broadcom Inc.,0.000000,1.371847
6,CVX,Chevron Corporation,0.000000,-0.962778
7,KO,Coca-Cola Company,0.763218,-0.421864
8,COST,Costco Wholesale Corporation,0.000000,-0.215102
9,LLY,Eli Lilly and Company,0.000000,-0.071729


# Profitability

In [89]:
# select the profitability metrics
profitability_df = df_imp[['EBITDA Margin', 'EBIT Margin', 'Net Income Margin', 'ROA', 'ROE','ROIC', 'Asset Turnover']]

# standardize the data
scaler = StandardScaler()
profitability_df_scaled = scaler.fit_transform(profitability_df)

# perform PCA on the standardized data
pca = PCA(n_components=1)
profitability_pca = pca.fit_transform(profitability_df_scaled)

df_2['Profitability'] = profitability_pca.flatten()
df_2

,Symbol,Name,Insider Buying,Return,Profitability
0,ABBV,"AbbVie, Inc.",0.000000,-0.130704,1.220049
1,GOOGL,Alphabet Inc. Class A,0.000000,1.067291,0.291659
2,AMZN,"Amazon.com, Inc.",0.000000,-0.529775,-3.277301
3,AAPL,Apple Inc.,0.000000,0.698126,1.717104
4,BAC,Bank of America Corp,0.000000,-3.012115,-0.471945
5,AVGO,Broadcom Inc.,0.000000,1.371847,3.316467
6,CVX,Chevron Corporation,0.000000,-0.962778,-1.020102
7,KO,Coca-Cola Company,0.763218,-0.421864,-0.029637
8,COST,Costco Wholesale Corporation,0.000000,-0.215102,-3.761259
9,LLY,Eli Lilly and Company,0.000000,-0.071729,0.230124


# Valuation 

In [90]:
# select the valuation metrics
valuation_df = df_imp[['Price /Sales CFY', 'Price / Earns Ratio','Price / Book Value', 'Entrpr Value/ Sales', 'Entrpr Value/ EBITDA', 'Entrpr Value/ EBIT']]

# standardize the data
scaler = StandardScaler()
valuation_df_scaled = scaler.fit_transform(valuation_df)

# perform PCA on the standardized data
pca = PCA(n_components=1)
valuation_pca = pca.fit_transform(valuation_df_scaled)

df_2['Valuation'] = valuation_pca.flatten()
df_2

,Symbol,Name,Insider Buying,Return,Profitability,Valuation
0,ABBV,"AbbVie, Inc.",0.000000,-0.130704,1.220049,-0.681141
1,GOOGL,Alphabet Inc. Class A,0.000000,1.067291,0.291659,-0.917850
2,AMZN,"Amazon.com, Inc.",0.000000,-0.529775,-3.277301,0.334216
3,AAPL,Apple Inc.,0.000000,0.698126,1.717104,0.025147
4,BAC,Bank of America Corp,0.000000,-3.012115,-0.471945,-0.757930
5,AVGO,Broadcom Inc.,0.000000,1.371847,3.316467,-0.590601
6,CVX,Chevron Corporation,0.000000,-0.962778,-1.020102,-2.174639
7,KO,Coca-Cola Company,0.763218,-0.421864,-0.029637,0.459793
8,COST,Costco Wholesale Corporation,0.000000,-0.215102,-3.761259,-0.290094
9,LLY,Eli Lilly and Company,0.000000,-0.071729,0.230124,2.935885


In [91]:
df_2['Valuation_inver'] = 10 - df_2['Valuation']
df_2

,Symbol,Name,Insider Buying,Return,Profitability,Valuation,Valuation_inver
0,ABBV,"AbbVie, Inc.",0.000000,-0.130704,1.220049,-0.681141,10.681141
1,GOOGL,Alphabet Inc. Class A,0.000000,1.067291,0.291659,-0.917850,10.917850
2,AMZN,"Amazon.com, Inc.",0.000000,-0.529775,-3.277301,0.334216,9.665784
3,AAPL,Apple Inc.,0.000000,0.698126,1.717104,0.025147,9.974853
4,BAC,Bank of America Corp,0.000000,-3.012115,-0.471945,-0.757930,10.757930
5,AVGO,Broadcom Inc.,0.000000,1.371847,3.316467,-0.590601,10.590601
6,CVX,Chevron Corporation,0.000000,-0.962778,-1.020102,-2.174639,12.174639
7,KO,Coca-Cola Company,0.763218,-0.421864,-0.029637,0.459793,9.540207
8,COST,Costco Wholesale Corporation,0.000000,-0.215102,-3.761259,-0.290094,10.290094
9,LLY,Eli Lilly and Company,0.000000,-0.071729,0.230124,2.935885,7.064115


# Safety

In [92]:
# select the safety metrics
safety_df = df_imp[['Div Yield', 'EBIT Interest Coverage', 'Current Ratio']]

# standardize the data
scaler = StandardScaler()
safety_df_scaled = scaler.fit_transform(safety_df)

# perform PCA on the standardized data
pca = PCA(n_components=1)
safety_pca = pca.fit_transform(safety_df_scaled)

df_2['Safety'] = safety_pca.flatten()
df_2

,Symbol,Name,Insider Buying,Return,Profitability,Valuation,Valuation_inver,Safety
0,ABBV,"AbbVie, Inc.",0.000000,-0.130704,1.220049,-0.681141,10.681141,-1.651358
1,GOOGL,Alphabet Inc. Class A,0.000000,1.067291,0.291659,-0.917850,10.917850,3.291870
2,AMZN,"Amazon.com, Inc.",0.000000,-0.529775,-3.277301,0.334216,9.665784,-0.013255
3,AAPL,Apple Inc.,0.000000,0.698126,1.717104,0.025147,9.974853,0.143906
4,BAC,Bank of America Corp,0.000000,-3.012115,-0.471945,-0.757930,10.757930,-1.131661
5,AVGO,Broadcom Inc.,0.000000,1.371847,3.316467,-0.590601,10.590601,-0.214663
6,CVX,Chevron Corporation,0.000000,-0.962778,-1.020102,-2.174639,12.174639,-0.230007
7,KO,Coca-Cola Company,0.763218,-0.421864,-0.029637,0.459793,9.540207,-1.063312
8,COST,Costco Wholesale Corporation,0.000000,-0.215102,-3.761259,-0.290094,10.290094,0.371192
9,LLY,Eli Lilly and Company,0.000000,-0.071729,0.230124,2.935885,7.064115,-0.163199


In [93]:
df_2.describe()

,Insider Buying,Return,Profitability,Valuation,Valuation_inver,Safety
count,29.000000,2.900000e+01,2.900000e+01,2.900000e+01,29.000000,2.900000e+01
mean,0.041217,6.316786e-17,1.531342e-17,2.297013e-17,10.000000,-7.656711e-18
std,0.149699,1.496112e+00,2.010809e+00,2.035837e+00,2.035837,1.314736e+00
min,0.000000,-3.012115e+00,-3.761259e+00,-2.331235e+00,1.437369,-1.651358e+00
25%,0.000000,-5.635251e-01,-1.020102e+00,-9.178501e-01,9.639421,-8.756068e-01
50%,0.000000,-2.066670e-01,2.002471e-01,-4.407411e-01,10.440741,-2.934027e-01
75%,0.000000,1.876862e-01,8.936375e-01,3.605794e-01,10.917850,3.711919e-01
max,0.763218,4.926694e+00,4.281837e+00,8.562631e+00,12.331235,3.995147e+00


# Final Factor Score Calculation

In [94]:
# assign weights to each dimension
w_insider_buying = 0.3
w_return = 0.2
w_profitability = 0.2
w_valuation_inver = 0.2
w_safety = 0.1

# calculate the weighted score for each stock
df_2['Weighted Score'] = (
    w_insider_buying * df_2['Insider Buying'] +
    w_return * df_2['Return'] +
    w_profitability * df_2['Profitability'] +
    w_valuation_inver * df_2['Valuation_inver'] +
    w_safety * df_2['Safety']
    
)

# sort the stocks by weighted score in descending order
df_2 = df_2.sort_values(by=['Weighted Score'], ascending=False)

df_2 = df_2.reset_index(drop=True)
df_2

,Symbol,Name,Insider Buying,Return,Profitability,Valuation,Valuation_inver,Safety,Weighted Score
0,META,Meta Platforms Inc. Class A,0.000000,3.699073,0.200247,-1.792437,11.792437,3.995147,3.537866
1,AVGO,Broadcom Inc.,0.000000,1.371847,3.316467,-0.590601,10.590601,-0.214663,3.034317
2,GOOGL,Alphabet Inc. Class A,0.000000,1.067291,0.291659,-0.917850,10.917850,3.291870,2.784547
3,V,Visa Inc. Class A,0.000000,0.187686,4.116316,1.266371,8.733629,0.455354,2.653062
4,MSFT,Microsoft Corporation,0.000000,1.098454,1.926478,0.746227,9.253773,0.738418,2.529583
5,AAPL,Apple Inc.,0.000000,0.698126,1.717104,0.025147,9.974853,0.143906,2.492407
6,MA,Mastercard Incorporated Class A,0.000000,-0.021304,4.281837,2.433661,7.566339,0.204152,2.385789
7,ORCL,Oracle Corporation,0.000000,0.564662,0.441858,-0.440741,10.440741,-0.293403,2.260112
8,MRK,"Merck & Co., Inc.",0.000000,-0.214728,0.792945,-0.735699,10.735699,-0.597327,2.203051
9,ABBV,"AbbVie, Inc.",0.000000,-0.130704,1.220049,-0.681141,10.681141,-1.651358,2.188962


In [95]:
ticker_list = df_2['Symbol'].tolist()
print(ticker_list)

['META', 'AVGO', 'GOOGL', 'V', 'MSFT', 'AAPL', 'MA', 'ORCL', 'MRK', 'ABBV', 'MCD', 'XOM', 'PFE', 'CVX', 'KO', 'PG', 'TMO', 'HD', 'JNJ', 'JPM', 'TSLA', 'NVDA', 'PEP', 'WMT', 'LLY', 'UNH', 'BAC', 'COST', 'AMZN']


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ee24e42d-4e53-4868-a1a5-ce3590710dbf' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>